In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

from sklearn import linear_model, svm, cross_validation, naive_bayes, ensemble, tree, neighbors, decomposition, preprocessing
from pandas.tseries.offsets import BDay

import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    
    exo_series = exo_series[exo_series.columns[0]]
    df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.tail()

In [ ]:
px_ser = df.NG_BearishCollarBW
px_ser.dropna().plot()

# Model

In [ ]:
i = 100

empty_ser = pd.Series(False, index=px_ser.dropna().index)

signals_df = pd.DataFrame()

signals_df['long_entry_signal'] = empty_ser
signals_df['long_exit_signal'] = empty_ser

signals_df['short_entry_signal'] = empty_ser
signals_df['short_exit_signal'] = empty_ser

while True:
    dataset_df = pd.DataFrame(index=px_ser.dropna().iloc[:i].index)
    
    px_ser_slice = px_ser[dataset_df.index].rolling(10).median()
    
    dataset_df['target'] = px_ser_slice.shift(-1)
    
    #dataset_df['features_px'] = px_ser_slice
    dataset_df['features_shift1'] = px_ser_slice.shift(1)
    dataset_df['features_shift2'] = px_ser_slice.shift(2)
    #dataset_df['features_shift3'] = px_ser_slice.shift(3)
    #dataset_df['features_shift5'] = px_ser_slice.shift(5)
    #dataset_df['features_shift10'] = px_ser_slice.shift(10)

    dataset_df['features_ma5'] = px_ser_slice.rolling(5).mean()
    #dataset_df['features_ma10'] = px_ser_slice.rolling(10).mean()
    #dataset_df['features_ma20'] = px_ser_slice.rolling(20).mean()
    #dataset_df['features_ma30'] = px_ser_slice.rolling(30).mean()

    dataset_df['features_relstr5'] = px_ser_slice - px_ser_slice.rolling(5).mean()
    #dataset_df['features_relstr10'] = px_ser_slice - px_ser_slice.rolling(10).mean()
    #dataset_df['features_relstr20'] = px_ser_slice - px_ser_slice.rolling(20).mean()
    #dataset_df['features_relstra30'] = px_ser_slice - px_ser_slice.rolling(30).mean()
    
    predict_features = dataset_df.filter(like='features').dropna()
    
    dataset_df.dropna(inplace=True)

    #dataset_df = pd.DataFrame(data=preprocessing.StandardScaler().fit_transform(dataset_df.values), 
    #                          index=dataset_df.index, columns=dataset_df.columns)

    #dataset_df = pd.DataFrame(data=preprocessing.RobustScaler().fit_transform(dataset_df.values), 
    #                          index=dataset_df.index, columns=dataset_df.columns)

    target = dataset_df.target

    features = dataset_df.filter(like='features')
    
    #rf = ensemble.RandomForestRegressor(n_estimators=50).fit(features.iloc[:i], target.iloc[:i])
    #features = features[pd.Series(rf.feature_importances_, index=features.columns).sort_values()[-3:].index]
    
    
    #model = linear_model.LinearRegression()
    #model = linear_model.ElasticNet()
    #model = linear_model.RANSACRegressor()
    
    #knn = neighbors.KNeighborsRegressor(5)
    #lr = linear_model.LinearRegression()
    #model = ensemble.BaggingRegressor(base_estimator=None, n_estimators=50)
    
    #model = linear_model.RANSACRegressor()
    #model = tree.DecisionTreeRegressor(max_depth=4)
    #model = svm.NuSVR()
    #model = ensemble.AdaBoostRegressor(n_estimators=10)
    #model = ensemble.GradientBoostingRegressor()
    #model = ensemble.RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)
    model = neighbors.KNeighborsRegressor(5, algorithm='auto')
    #model = neighbors.RadiusNeighborsRegressor(5, n_jobs=1, algorithm='auto')
    
    model.fit(features.iloc[:i], target.iloc[:i])
        
    '''if i <= px_ser.size/4:
        model = neighbors.KNeighborsRegressor(5, n_jobs=-1)
        model.fit(features.iloc[:i], target.iloc[:i])
        print('NO FITTING FROM',  px_ser[dataset_df.index].iloc[:i].index[-1])'''
    
    '''if prediction_series.size == 0:    
        prediction_series = pd.Series(model.predict(predict_features.iloc[:i]), index=predict_features.iloc[:i].index, name='Prediction')
    
    else:
        prediction_series = pd.concat([prediction_series,
                                       pd.Series(model.predict(predict_features.iloc[:i])[-1], 
                                                 index=[predict_features.iloc[:i].index[-1] + pd.DateOffset(1)])])'''
    
        
    prediction_series = pd.Series(model.predict(features.iloc[:i]), 
                                  index=features.iloc[:i].index, name='Prediction') 
    
    #if prediction_series[-1] > prediction_series.rolling(5).mean()[-1]:
    if prediction_series[-1] > prediction_series[-2]:
        signals_df['long_entry_signal'].ix[px_ser_slice.index[-1]] = True
        
        
        signals_df['long_exit_signal'].ix[px_ser_slice.index[-1] + BDay(1)] = True
            
           
    #if prediction_series[-1] < prediction_series.rolling(5).mean()[-1]:
    if prediction_series[-1] < prediction_series[-2]:
        signals_df['short_entry_signal'].ix[px_ser_slice.index[-1]] = True
        
        
        signals_df['short_exit_signal'].ix[px_ser_slice.index[-1]+ BDay(1)] = True
            
    print('last date-', prediction_series.iloc[:i].index[-1])
    
    if i >= px_ser.dropna().size:
        break
        
    i += 1
    
#signals_df.loc[(signals_df.long_exit_signal == True) & (signals_df.long_entry_signal == True), 'long_exit_signal'] = False
#signals_df.loc[(signals_df.short_exit_signal == True) & (signals_df.short_entry_signal == True), 'short_exit_signal'] = False

# Backtest

In [ ]:
entry = signals_df.long_entry_signal
exit = signals_df.long_exit_signal

#entry = signals_df.short_entry_signal
#exit = signals_df.short_exit_signal

exodf = pd.DataFrame()
exodf['exo'] = px_ser.dropna()

direction = 1
pl, inposition = backtester.backtest(exodf.iloc[:], entry, exit, direction)
equity, stats = backtester.stats(pl, inposition)

equity.plot()
(px_ser*-1).plot()

stats

In [ ]:
#entry = signals_df.long_entry_signal
#exit = signals_df.long_exit_signal

entry = signals_df.short_entry_signal
exit = signals_df.short_exit_signal

exodf = pd.DataFrame()
exodf['exo'] = px_ser.dropna()

direction = -1
pl, inposition = backtester.backtest(exodf.iloc[:], entry, exit, direction)
equity, stats = backtester.stats(pl, inposition)

equity.plot()

(px_ser*-1).plot()

stats